In [1]:
import os
import pandas as pd
import polars as pl
import numpy as np
# import validation_data_input
import plotly.express as px
import toml
import psrc_theme
from pathlib import Path
import util

# to show plotly figures in quarto HTML file
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [2]:
config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'validation_configuration.toml'))
input_config = toml.load(Path(Path.cwd(), '..\..\..\..\configuration', 'input_configuration.toml'))

# try:
#   %store -r data
# except:
#   print("no data in memory")
data = util.ValidationData(config,input_config,['tour'])

In [3]:

mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: walk to transit",
            7: "7: park-and-ride",
            8: "8: school bus",
            9: "9:tnc"}
pdpurp_cat = {1: "1: Work",
              2: "2: School",
              3: "3: Escort",
              4: "4: other home-based",
              5: "4: other home-based",
              6: "4: other home-based",
              7: "4: other home-based",
              8: "4: other home-based",
              9: "4: other home-based",
              10: "4: other home-based"}
df_tour = data.tour.with_columns(
    tmodetp_label=pl.col("tmodetp").replace_strict(mode_cat, default=None),
    pdpurp_label=pl.col("pdpurp").replace_strict(pdpurp_cat, default=None))

In [4]:
def plot_mode_choice(df: pl.DataFrame, title_name: str):
    df_plot = (
        df.group_by(['source', 'pdpurp_label', 'tmodetp_label'])
        .agg(
            toexpfac_sum=pl.col('toexpfac').sum(),
            sample_count=pl.col('toexpfac').count()
        )
        .with_columns(
            (pl.col('toexpfac_sum') / pl.col('toexpfac_sum').sum().over(['source', 'pdpurp_label'])).alias('percentage')
        )
    )

    fig = px.bar(
        df_plot.sort(by=['pdpurp_label', 'source']),
        x="tmodetp_label",
        y="percentage",
        color="source",
        facet_col='pdpurp_label',
        facet_col_wrap=2,
        barmode="group",
        hover_data=['sample_count'],
        title=title_name
    )
    fig.update_layout(
        height=600,
        width=900,
        font=dict(size=11),
        xaxis=dict(dtick=1, categoryorder='category ascending')
    )
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_yaxis(lambda a: a.update(tickformat=".1%"))
    fig.show()

In [5]:
# plot mode choice by purpose
all_tour = df_tour.filter(pl.col('parent') == 0)
plot_mode_choice(all_tour,"tour mode choice by purpose")